In [73]:
#data manipulation & storage
import pandas as pd
import numpy as np
#visualization
import seaborn as sb
from matplotlib import pyplot as plt
#performance and preprocessing
from sklearn import metrics, model_selection, preprocessing
from sklearn.utils import resample
#models
from sklearn import ensemble, tree, neighbors, naive_bayes, linear_model

In [74]:
def get_means(results_dictionary):
    '''
    Returns average of all keys inside dictionary. 
    results_dictionary must have keys with numeric values
    '''
    if type(results_dictionary) != dict: return None
    average_dict = {}
    for key, val in results_dictionary.items():
        key += ' average'
        average_dict[key] = val.mean()
    return average_dict

In [75]:
def print_dictionary_elts(dictionary):
    '''
    Pretty print key/value pairs of a dictionary
    '''
    for key,val in dictionary.items():
        print(f'{key}: {val}')

In [76]:
def run_model(model, x, y, score, cv):
    '''Run cross-validation on sklearn model
    Arguments:
        model - model to be run using cross-validation
        x - vector of independent variables/features
        y - vector of dependent variables/outcomes
        score - scoring metric(s) to estimate model performance
        cv - sklearn KFold cross-validation object
    Returns:
        dictionary with scoring metric summary based on specified cross-validation
    '''
    return model_selection.cross_validate(
        model,
        x,
        y,
        cv=cv,
        scoring=score)

In [77]:
cv = model_selection.KFold(n_splits=10, random_state=42, shuffle=True)

In [78]:
#scoring = ('accuracy', 'f1_weighted', 'roc_auc_ovo_weighted')

In [79]:
scoring = ('f1_weighted', 'accuracy')

In [80]:
df = pd.read_csv('merged_info_volcanoes.csv')

In [81]:
df

,Volcano Number,Volcano Name,VEI,Eruption Category,Latitude,Longitude,Country,Recoded Volcano Type,Region,Recoded Dominant Rock Type,Tectonic Setting,Elevation (m)
0,264050,Sangeang Api,NaN,Confirmed Eruption,-8.200,119.070,Indonesia,Complex,Indonesia,Trachybasalt / Tephrite Basanite,Subduction zone / Continental crust (>25 km),1912
1,264050,Sangeang Api,2.0,Confirmed Eruption,-8.200,119.070,Indonesia,Complex,Indonesia,Trachybasalt / Tephrite Basanite,Subduction zone / Continental crust (>25 km),1912
2,264050,Sangeang Api,3.0,Confirmed Eruption,-8.200,119.070,Indonesia,Complex,Indonesia,Trachybasalt / Tephrite Basanite,Subduction zone / Continental crust (>25 km),1912
3,264050,Sangeang Api,NaN,Uncertain Eruption,-8.200,119.070,Indonesia,Complex,Indonesia,Trachybasalt / Tephrite Basanite,Subduction zone / Continental crust (>25 km),1912
4,264050,Sangeang Api,2.0,Confirmed Eruption,-8.200,119.070,Indonesia,Complex,Indonesia,Trachybasalt / Tephrite Basanite,Subduction zone / Continental crust (>25 km),1912
...,...,...,...,...,...,...,...,...,...,...,...,...
11109,327812,Red Hill,NaN,Confirmed Eruption,34.250,-108.830,United States,Volcanic field,Canada and Western USA,Basalt / Picro-Basalt,Rift zone / Continental crust (>25 km),2300
11110,327812,Red Hill,NaN,Confirmed Eruption,34.250,-108.830,United States,Volcanic field,Canada and Western USA,Basalt / Picro-Basalt,Rift zone / Continental crust (>25 km),2300
11111,327812,Red Hill,NaN,Confirmed Eruption,34.250,-108.830,United States,Volcanic field,Canada and Western USA,Basalt / Picro-Basalt,Rift zone / Continental crust (>25 km),2300
11112,283141,Nantaisan,NaN,Confirmed Eruption,36.765,139.491,Japan,Stratovolcano,"Japan, Taiwan, Marianas",Andesite / Basaltic Andesite,Subduction zone / Continental crust (>25 km),2486


In [82]:
null_count =  df.VEI.isnull().sum()
print(f'There are {null_count} observations missing VEI.')

There are 2831 observations missing VEI.


In [83]:
df.VEI.value_counts()

2.0    4012
1.0    1383
3.0    1157
0.0     987
4.0     508
5.0     177
6.0      52
7.0       7
Name: VEI, dtype: int64

In [84]:
#drop observations with null VEI
bad_indices, vei_values, idx = [], df.VEI.values, 0
for val in vei_values:
    if np.isnan(val): bad_indices.append(idx)
    idx += 1
#print(len(bad_indices))
df.drop(index=bad_indices, inplace=True)

In [85]:
df.shape

(8283, 12)

In [86]:
null_count =  df.VEI.isnull().sum()
print(f'There are {null_count} observations missing VEI.')

There are 0 observations missing VEI.


In [87]:
ind_cols = [
    #'Volcano Number',
    'Latitude',
    'Longitude',
    'Country',
    'Recoded Volcano Type',
    'Region',
    'Recoded Dominant Rock Type',
    'Tectonic Setting',
    'Elevation (m)'
]

In [88]:
dep_cols = [
    #'Volcano Number',
    'VEI'
]

In [89]:
scaler = preprocessing.MinMaxScaler()

In [90]:
# Get indendent variable columns
x = df[ind_cols]
x.shape

(8283, 8)

In [91]:
elevation = x['Elevation (m)'].values
x['Elevation'] = scaler.fit_transform(elevation.reshape(-1, 1))

<ipython-input-91-ef547e4caa2c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Elevation'] = scaler.fit_transform(elevation.reshape(-1, 1))


In [92]:
latitude = x['Latitude'].values
x['Latitude'] = scaler.fit_transform(latitude.reshape(-1,1))

<ipython-input-92-23efc708c28c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Latitude'] = scaler.fit_transform(latitude.reshape(-1,1))


In [93]:
longitude = x['Longitude'].values
x['Longitude'] = scaler.fit_transform(longitude.reshape(-1,1))

<ipython-input-93-c46fa3ce87d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Longitude'] = scaler.fit_transform(longitude.reshape(-1,1))


In [94]:
x= x.drop(['Elevation (m)'], axis=1)
x

,Latitude,Longitude,Country,Recoded Volcano Type,Region,Recoded Dominant Rock Type,Tectonic Setting,Elevation
1,0.424978,0.831706,Indonesia,Complex,Indonesia,Trachybasalt / Tephrite Basanite,Subduction zone / Continental crust (>25 km),0.605136
2,0.424978,0.831706,Indonesia,Complex,Indonesia,Trachybasalt / Tephrite Basanite,Subduction zone / Continental crust (>25 km),0.605136
4,0.424978,0.831706,Indonesia,Complex,Indonesia,Trachybasalt / Tephrite Basanite,Subduction zone / Continental crust (>25 km),0.605136
5,0.424978,0.831706,Indonesia,Complex,Indonesia,Trachybasalt / Tephrite Basanite,Subduction zone / Continental crust (>25 km),0.605136
6,0.424978,0.831706,Indonesia,Complex,Indonesia,Trachybasalt / Tephrite Basanite,Subduction zone / Continental crust (>25 km),0.605136
...,...,...,...,...,...,...,...,...
11103,0.798281,0.940759,Russia,Stratovolcano,Kamchatka and Mainland Asia,Andesite / Basaltic Andesite,Subduction zone / Continental crust (>25 km),0.625884
11104,0.780811,0.158448,Canada,Stratovolcano,Canada and Western USA,Dacite,Subduction zone / Continental crust (>25 km),0.666031
11107,0.009072,0.122292,Antarctica,Shield,Antarctica,Trachyte / Trachydacite,Intraplate / Continental crust (>25 km),0.729629
11108,0.739944,0.182500,United States,Shield,Canada and Western USA,Basalt / Picro-Basalt,Rift zone / Continental crust (>25 km),0.570634


In [95]:
x = pd.get_dummies(x)
x

,Latitude,Longitude,Elevation,Country_Antarctica,Country_Argentina,Country_Armenia,Country_Australia,Country_Burma (Myanmar),Country_Cameroon,Country_Canada,...,Tectonic Setting_Intraplate / Continental crust (>25 km),Tectonic Setting_Intraplate / Intermediate crust (15-25 km),Tectonic Setting_Intraplate / Oceanic crust (< 15 km),Tectonic Setting_Rift zone / Continental crust (>25 km),Tectonic Setting_Rift zone / Intermediate crust (15-25 km),Tectonic Setting_Rift zone / Oceanic crust (< 15 km),Tectonic Setting_Subduction zone / Continental crust (>25 km),Tectonic Setting_Subduction zone / Crustal thickness unknown,Tectonic Setting_Subduction zone / Intermediate crust (15-25 km),Tectonic Setting_Subduction zone / Oceanic crust (< 15 km)
1,0.424978,0.831706,0.605136,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0.424978,0.831706,0.605136,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0.424978,0.831706,0.605136,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0.424978,0.831706,0.605136,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,0.424978,0.831706,0.605136,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,0.798281,0.940759,0.625884,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
11104,0.780811,0.158448,0.666031,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
11107,0.009072,0.122292,0.729629,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
11108,0.739944,0.182500,0.570634,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [96]:
x.describe()

,Latitude,Longitude,Elevation,Country_Antarctica,Country_Argentina,Country_Armenia,Country_Australia,Country_Burma (Myanmar),Country_Cameroon,Country_Canada,...,Tectonic Setting_Intraplate / Continental crust (>25 km),Tectonic Setting_Intraplate / Intermediate crust (15-25 km),Tectonic Setting_Intraplate / Oceanic crust (< 15 km),Tectonic Setting_Rift zone / Continental crust (>25 km),Tectonic Setting_Rift zone / Intermediate crust (15-25 km),Tectonic Setting_Rift zone / Oceanic crust (< 15 km),Tectonic Setting_Subduction zone / Continental crust (>25 km),Tectonic Setting_Subduction zone / Crustal thickness unknown,Tectonic Setting_Subduction zone / Intermediate crust (15-25 km),Tectonic Setting_Subduction zone / Oceanic crust (< 15 km)
count,8283.000000,8283.000000,8283.000000,8283.000000,8283.000000,8283.000000,8283.000000,8283.000000,8283.000000,8283.000000,...,8283.000000,8283.000000,8283.000000,8283.000000,8283.000000,8283.000000,8283.000000,8283.000000,8283.000000,8283.000000
mean,0.560899,0.613501,0.622253,0.003622,0.000483,0.000121,0.002052,0.000121,0.002294,0.000724,...,0.010986,0.001328,0.058674,0.018592,0.002898,0.061572,0.650610,0.053966,0.047447,0.093807
std,0.183284,0.317051,0.112818,0.060077,0.021971,0.010988,0.045260,0.010988,0.047842,0.026906,...,0.104245,0.036420,0.235028,0.135088,0.053754,0.240391,0.476806,0.225964,0.212605,0.291577
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.429023,0.291047,0.550282,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.556952,0.798089,0.605136,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.706629,0.888600,0.684474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [97]:
# Get dependent variable columns
y = df[dep_cols]
y.shape

(8283, 1)

In [98]:
# split data into train and validation sets.  Validation will be held out to estimate model generalizability
x_train, x_val, y_train, y_val = \
    model_selection.train_test_split(
        x,
        y,
        test_size = 0.30,
        random_state = 42) 

In [99]:
x_train.shape

(5798, 125)

In [100]:
y_train.shape

(5798, 1)

In [101]:
x_val.shape

(2485, 125)

In [102]:
y_val.shape

(2485, 1)

In [103]:
y_train.value_counts()

VEI
2.0    2782
1.0     966
3.0     804
0.0     712
4.0     360
5.0     129
6.0      39
7.0       6
dtype: int64

In [104]:
y_val.value_counts()

VEI
2.0    1230
1.0     417
3.0     353
0.0     275
4.0     148
5.0      48
6.0      13
7.0       1
dtype: int64

In [105]:
np.ravel(y_train)

array([2., 3., 1., ..., 2., 2., 4.])

In [106]:
# Bring training data back together
training_data = pd.concat([x_train, y_train], axis=1)
training_data

,Latitude,Longitude,Elevation,Country_Antarctica,Country_Argentina,Country_Armenia,Country_Australia,Country_Burma (Myanmar),Country_Cameroon,Country_Canada,...,Tectonic Setting_Intraplate / Intermediate crust (15-25 km),Tectonic Setting_Intraplate / Oceanic crust (< 15 km),Tectonic Setting_Rift zone / Continental crust (>25 km),Tectonic Setting_Rift zone / Intermediate crust (15-25 km),Tectonic Setting_Rift zone / Oceanic crust (< 15 km),Tectonic Setting_Subduction zone / Continental crust (>25 km),Tectonic Setting_Subduction zone / Crustal thickness unknown,Tectonic Setting_Subduction zone / Intermediate crust (15-25 km),Tectonic Setting_Subduction zone / Oceanic crust (< 15 km),VEI
3920,0.422881,0.842011,0.588521,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,2.0
9326,0.432480,0.859725,0.511010,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3.0
7101,0.795921,0.020748,0.536927,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1.0
7849,0.546225,0.618662,0.476826,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1.0
9079,0.212893,0.299054,0.648064,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6977,0.697305,0.883207,0.648303,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2.0
6178,0.563958,0.247782,0.752286,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2.0
6431,0.867444,0.445846,0.571588,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2.0
958,0.243193,0.302564,0.687893,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2.0


In [107]:
# Since data is imbalanced (counts of outcome classes are very different)
# try upsampling the minority classes.  Note that majority class is
# that where VEI = 2.0
vei_0 = training_data[training_data.VEI == 0.0]
vei_1 = training_data[training_data.VEI == 1.0]
vei_2 = training_data[training_data.VEI == 2.0]
vei_3 = training_data[training_data.VEI == 3.0]
vei_4 = training_data[training_data.VEI == 4.0]
vei_5 = training_data[training_data.VEI == 5.0]
vei_6 = training_data[training_data.VEI == 6.0]
vei_7 = training_data[training_data.VEI == 7.0]

In [108]:
vei_0_upsample = resample(vei_0,
                          replace=True, #sample w/ replacement
                          n_samples = len(vei_2), #match majority class count
                          random_state = 42)
vei_0_upsample.shape

(2782, 126)

In [109]:
vei_1_upsample = resample(vei_1,
                          replace=True, #sample w/ replacement
                          n_samples = len(vei_2), #match majority class count
                          random_state = 42)
vei_1_upsample.shape

(2782, 126)

In [110]:
vei_3_upsample = resample(vei_3,
                          replace=True, #sample w/ replacement
                          n_samples = len(vei_2), #match majority class count
                          random_state = 42)
vei_3_upsample.shape

(2782, 126)

In [111]:
vei_4_upsample = resample(vei_4,
                          replace=True, #sample w/ replacement
                          n_samples = len(vei_2), #match majority class count
                          random_state = 42)
vei_4_upsample.shape

(2782, 126)

In [112]:
vei_5_upsample = resample(vei_5,
                          replace=True, #sample w/ replacement
                          n_samples = len(vei_2), #match majority class count
                          random_state = 42)
vei_5_upsample.shape

(2782, 126)

In [113]:
vei_6_upsample = resample(vei_6,
                          replace=True, #sample w/ replacement
                          n_samples = len(vei_2), #match majority class count
                          random_state = 42)
vei_6_upsample.shape

(2782, 126)

In [114]:
vei_7_upsample = resample(vei_7,
                          replace=True, #sample w/ replacement
                          n_samples = len(vei_2), #match majority class count
                          random_state = 42)
vei_7_upsample.shape

(2782, 126)

In [115]:
# Now, combine all of the upsampled classes again
classes = [
    vei_0_upsample,
    vei_1_upsample,
    vei_2,
    vei_3_upsample,
    vei_4_upsample,
    vei_5_upsample,
    vei_6_upsample,
    vei_7_upsample
]
upsampled = pd.concat(classes)
upsampled.shape

(22256, 126)

In [116]:
y_train = upsampled.VEI

In [117]:
x_train = upsampled.drop('VEI', axis=1)

In [118]:
y_train.shape

(22256,)

In [119]:
x_train.shape

(22256, 125)

In [120]:
## Now, construct models

In [121]:
# Figure out the best K for KNN
knn_cv = model_selection.KFold(n_splits=5, random_state=42, shuffle=True)
best_n, best_acc = None, 0
for n in range(40):
    n += 1
    knn = neighbors.KNeighborsClassifier(n_neighbors = n)
    results = get_means(run_model(knn, x_train, np.ravel(y_train), scoring, knn_cv))
    acc = results['test_accuracy average']
    print(f'Number of neighbors is {n}... & accuracy is {acc}')
    if acc > best_acc:
        best_n, best_acc = n, acc

Number of neighbors is 1... & accuracy is 0.3507820809319731
Number of neighbors is 2... & accuracy is 0.3492094309141994
Number of neighbors is 3... & accuracy is 0.3650254654707756
Number of neighbors is 4... & accuracy is 0.612823854356603
Number of neighbors is 5... & accuracy is 0.6157440517823811
Number of neighbors is 6... & accuracy is 0.6189340029386574
Number of neighbors is 7... & accuracy is 0.6261234591376642
Number of neighbors is 8... & accuracy is 0.6225735537386937
Number of neighbors is 9... & accuracy is 0.6283698223018621
Number of neighbors is 10... & accuracy is 0.6304815155058688
Number of neighbors is 11... & accuracy is 0.6295828107719013
Number of neighbors is 12... & accuracy is 0.6297624245477812
Number of neighbors is 13... & accuracy is 0.6290884590781158
Number of neighbors is 14... & accuracy is 0.6283245756982844
Number of neighbors is 15... & accuracy is 0.6292230684807295
Number of neighbors is 16... & accuracy is 0.6293579705783027
Number of neighbor

In [122]:
print(f'Best n is {best_n} and best accuracy is {best_acc}')

Best n is 10 and best accuracy is 0.6304815155058688


In [123]:
lr = linear_model.LogisticRegression(
    penalty='l2',
    C=0.5,
    random_state=42,
    multi_class='multinomial',
    max_iter = 1000
)
lr

LogisticRegression(C=0.5, max_iter=1000, multi_class='multinomial',
                   random_state=42)

In [124]:
nb = naive_bayes.MultinomialNB(
    alpha=5.0,
    fit_prior=True)
nb

MultinomialNB(alpha=5.0)

In [125]:
knn = neighbors.KNeighborsClassifier(n_neighbors = best_n)
knn

KNeighborsClassifier(n_neighbors=10)

In [126]:
tr = tree.DecisionTreeClassifier(
    random_state=42,
    min_samples_leaf=5)
tr

DecisionTreeClassifier(min_samples_leaf=5, random_state=42)

In [127]:
rf = ensemble.RandomForestClassifier(
    n_estimators=1000,
    min_samples_leaf=5,
    random_state=42)
rf

RandomForestClassifier(min_samples_leaf=5, n_estimators=1000, random_state=42)

In [128]:
gbt = ensemble.GradientBoostingClassifier(
    n_estimators=500,
    min_samples_leaf=5,
    random_state=42,
    warm_start=True)
gbt

GradientBoostingClassifier(min_samples_leaf=5, n_estimators=500,
                           random_state=42, warm_start=True)

In [129]:
lr_results = run_model(lr, x_train, np.ravel(y_train), scoring, cv)
get_means(lr_results)

{'fit_time average': 11.519241333007812,
 'score_time average': 0.011588978767395019,
 'test_f1_weighted average': 0.4864351397657759,
 'test_accuracy average': 0.5062457776835559}

In [130]:
nb_results = run_model(nb, x_train, np.ravel(y_train), scoring, cv)
get_means(nb_results)

{'fit_time average': 0.0803959846496582,
 'score_time average': 0.014109611511230469,
 'test_f1_weighted average': 0.41638255360263343,
 'test_accuracy average': 0.4451825312698749}

In [131]:
knn_results = run_model(knn, x_train, np.ravel(y_train), scoring, cv)
means = get_means(knn_results)
means

{'fit_time average': 0.04988188743591308,
 'score_time average': 1.8658242702484131,
 'test_f1_weighted average': 0.6190683666370818,
 'test_accuracy average': 0.6279207325075462}

In [132]:
tr_results = run_model(tr, x_train, np.ravel(y_train), scoring, cv)
get_means(tr_results)

{'fit_time average': 0.34269371032714846,
 'score_time average': 0.013712310791015625,
 'test_f1_weighted average': 0.6445743468077669,
 'test_accuracy average': 0.6569919137466307}

In [133]:
rf_results = run_model(rf, x_train, np.ravel(y_train), scoring, cv)
get_means(rf_results)

{'fit_time average': 40.99273099899292,
 'score_time average': 0.865235424041748,
 'test_f1_weighted average': 0.6421830959473673,
 'test_accuracy average': 0.6541162361064842}

In [134]:
gbt_results = run_model(gbt, x_train, np.ravel(y_train), scoring, cv)
get_means(gbt_results)

{'fit_time average': 280.5717463493347,
 'score_time average': 0.1769176483154297,
 'test_f1_weighted average': 0.6486225104784504,
 'test_accuracy average': 0.6604513159090221}